In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder

iris_data = load_iris() 
x = iris_data.data
y_ = iris_data.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)

x_train, x_test, y_train, y_test = train_test_split(x,y_,test_size=0.2)
y_train = encoder.fit_transform(y_train)

/home/grumptitan/.local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [7]:
hidden_1 = 10
hidden_2 = 10
input_size = 4
output_size = 3
np.random.seed(40)

def relu(x):
    x[x<0]=0.0
    return x

def diff_relu(x):
    x[x>0]=1.0
    x[x<=0]=0.0
    return x

def softmax(x):
    x = x/np.max(x)
    return np.exp(x)/np.sum(np.exp(x))

# Glorot initialization

lim1 = (6.0 / (4.0 + 10.0))**0.5
lim2 = (6.0 / (10.0 + 10.0))**0.5
lim3 = (6.0 / (10.0 + 3.0))**0.5

blim1 = (6.0 / (1.0 + 10.0))**0.5
blim2 = (6.0 / (1.0 + 10.0))**0.5
blim3 = (6.0 / (1.0 + 3.0))**0.5

In [15]:
np.random.seed(42)
W1 = np.random.uniform(-lim1,lim1,(hidden_1,input_size))
Bi1 = np.random.uniform(-blim1,blim1,(hidden_1,1))
W2 = np.random.uniform(-lim2,lim2,(hidden_2,hidden_1))
Bi2 = np.random.uniform(-blim2,blim2,(hidden_2,1))
W3 = np.random.uniform(-lim3,lim3,(output_size,hidden_2))
Bi3 = np.random.uniform(-blim3,blim3,(output_size,1))

epochs = 1
lr = 1e-9
for i in range(epochs):
    x,y_train = shuffle(x_train,y_train,random_state = 42)
    loss = 0
    for j in range(len(x)):
        w1 = w2 = w3 = 0
        b1 = b2 = b3 = 0
        #forward pass
        t = x[j]
        t = t[..., np.newaxis]
        hid1 = layer(t,W1,Bi1)
        rel_hid1 = relu(hid1.reshape(-1,1))
        hid2 = layer(rel_hid1,W2,Bi2)
        rel_hid2  = relu(hid2.reshape(-1,1))
        out = layer(rel_hid2,W3,Bi3)
        y = softmax(out.reshape(-1,1))
        #BACKPROPAGATION
        del1 = y - y_train[j].reshape(y.shape)
        del1 = del1.reshape(y.shape)
        s1 = np.matmul(W3.T,del1) * diff_relu(hid2)
        s2 = np.matmul(W2.T,s1) * diff_relu(hid1)
        
        b3 = del1.reshape(Bi3.shape)
        w3 = np.matmul(del1,rel_hid2.T)
        b2 = s1.reshape(Bi2.shape)
        w2 = np.matmul(s1,rel_hid1.T)
        b1 = s2.reshape(Bi1.shape)
        w1 = np.matmul(s2,(t.reshape(-1,1)).T)
        
        loss -= np.log(y[np.argmax(y_train[j])])
        
        W1 -= lr*w1
        W2 -= lr*w2
        W3 -= lr*w3
        Bi1 -= lr*b1
        Bi2 -= lr*b2
        Bi3 -= lr*b3
        
    if i%50 == 0:
        print(i,loss)
    loss = 0

0 [159.8321768]


In [10]:
y_pred = []
for j in range(len(x_test)):
    hid1 = layer(x_test[j],W1,Bi1)
    rel_hid1 = relu(hid1)
    hid2 = layer(rel_hid1,W2,Bi2)
    rel_hid2  = relu(hid2)
    out = layer(rel_hid2,W3,Bi3)
    y = softmax(out)
    y_pred.append(y)

In [13]:
a = np.array(y_pred)
a = np.squeeze(a)
eww = []
for i in a:
    eww.append(np.argmax(i))
print(eww)
print(y_test)
print(accuracy_score(y_test, eww))

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
(30, 3, 10)
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets